
2. O Conway’s Game of Life é um exemplo bastante conhecido de um autómato celular . Neste problema vamos modificar as regras do autómato da seguinte forma

    1. O espaço de estados é finito definido por uma grelha de células booleanas (morta=0/viva=1) de dimensão $\,N\times N\,$ (com $N>3$) identificadas por índices $\,(i,j)\in \{1..N\}$.  Estas $\;N^2\;$ células são aqui referidas como “normais”. 
        No estado inicial todas as células normais estão mortas excepto  um quadrado $\,3\times 3\,$, designado por “centro”, aleatoriamente posicionado formado apenas por células vivas.
    2. Adicionalmente existem $\,2\,N+1\,$ “células da borda” que correspondem a um dos índices, $i$ ou $j$, ser zero. As células da borda têm valores constantes que, no estado inicial, são gerados aleatoriamente com uma probabilidade $\,\rho\,$ de estarem vivas.
    3. As células normais o autómato modificam o estado de acordo com a regra “B3/S23”: i.e. a célula nasce (passa de $0$ a $1$) se tem exatamente 3 vizinhos vivos e sobrevive (mantém-se viva) se o número de vizinhos vivos é 2 ou 3, caso contrário morre ou continua morta.

A célula $\;(i_0,j_0)\;$e $\,(i_1,j_1)\,$ são vizinhas sse $\;(i_0-i_1=\pm 1) \,\lor\, (j_0-j_1 = \pm 1)$

Pretende-se:

Construir uma máquina de estados finita que represente este autómato; são parâmetros do problema os parâmetros $\,N,\rho\,$ e a posição do  “centro”.

Verificar se se conseguem provar as seguintes propriedades:
   1. Todos os estados acessíveis contém pelo menos uma célula viva.
   2. Toda a célula normal está viva pelo menos uma vez em algum estado acessível.


In [1]:
from pysmt.shortcuts import *
from z3 import *

In [15]:
def declare(t, N):
    state = {}
    state['pc'] = Int('pc'+str(t))
    
    state['pos'] = {}
    for i in range(N):
        for j in range(N):
            state['pos'][f'({i},{j})'] = Bool(f'pos({i},{j})'+str(t))


    return state

In [34]:
'''
from pyprobs import Probability as pr
probabilidade de sair True = 25%
pr.prob("25%", num=5)
[False, False, True, False, False]
'''
from pyprobs import Probability as pr
from random import randint

def init(state, prob, N):
    init_state = {}

    center_square = (randint(1, N-2), randint(1, N-2))
    
    
    # colocar o quadrado a True
    for i in range(center_square[0]-1, center_square[0]+2):
        for j in range(center_square[1]-1, center_square[1]+2):
            init_state[(i,j)] = (state['pos'][f'({i},{j})'] == True)
    
    
    # colocar tudo o que não é borda a falso
    for i in range(1, N):
        for j in range(1, N):
            if (i,j) not in init_state:
                init_state[(i,j)] = (state['pos'][f'({i},{j})'] == False)
    
      
    # border-x
    for i in range(0,1):
        for j in range(0, N):
            if (i,j) not in init_state:
                init_state[(i,j)]=(state['pos'][f'({i},{j})'] == pr.prob(prob, num=1))
                
    # border-y
    for i in range(0,N):
        for j in range(0, 1):
            if (i,j) not in init_state:
                init_state[(i,j)]=(state['pos'][f'({i},{j})'] == pr.prob(prob, num=1))
                

    statements = list(init_state.values())
    
    print(center_square)
    print(And(statements))
    
    return And(
        state['pc'] == 0,
        And(statements)
    )



In [35]:
def teste():
    t = {
        (0,0): Bool('x') == True,
        (1,0): Bool('y'),
        (2,0): Bool('z')
    }
    
    print(list(t.values()))
    
    lista = []
    lista.append(t[(0,0)])
    lista.append(t[(1,0)] == False)
    lista.append(t[(2,0)] == True)
    
    return lista
teste()

[x == True, y, z]


[x == True, y == False, z == True]

In [36]:
# Pensar num modelo para as transições

def gera_traco(declare,init,k, N):

    s = Solver()
    
    trace = [declare(i, N) for i in range(k)]

    # adicionar o estado inicial
    s.add(init(trace[0],"0%",N))
    
    # adicionar as transições
    #for i in range(k - 1):
    #    s.add(trans(trace[i], trace[i+1], N))
    
    
    check = s.check()
    if check == sat:
        m = s.model()
        for i in range(k):
            print("Passo ", i)
            for v in trace[i]:
                x = 0
                #print(v, "=", m[trace[i][v]])
            print("----------------")
    else:
        print(check)
                
gera_traco(declare,init,20, 5)

(1, 1)
And(pos(0,0)0 == True,
    pos(0,1)0 == True,
    pos(0,2)0 == True,
    pos(1,0)0 == True,
    pos(1,1)0 == True,
    pos(1,2)0 == True,
    pos(2,0)0 == True,
    pos(2,1)0 == True,
    pos(2,2)0 == True,
    pos(1,3)0 == False,
    pos(1,4)0 == False,
    pos(2,3)0 == False,
    pos(2,4)0 == False,
    pos(3,1)0 == False,
    pos(3,2)0 == False,
    pos(3,3)0 == False,
    pos(3,4)0 == False,
    pos(4,1)0 == False,
    pos(4,2)0 == False,
    pos(4,3)0 == False,
    pos(4,4)0 == False,
    pos(0,3)0 == False,
    pos(0,4)0 == False,
    pos(3,0)0 == False,
    pos(4,0)0 == False)
Passo  0
----------------
Passo  1
----------------
Passo  2
----------------
Passo  3
----------------
Passo  4
----------------
Passo  5
----------------
Passo  6
----------------
Passo  7
----------------
Passo  8
----------------
Passo  9
----------------
Passo  10
----------------
Passo  11
----------------
Passo  12
----------------
Passo  13
----------------
Passo  14
----------------
Passo  